In [ ]:
"""
Author: Ngo Van Uc
Date:
Contact: ngovanuc.1508@gmail.com
"""

'\nAuthor: Ngo Van Uc\nDate: \nContact: ngovanuc.1508@gmail.com\n'

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [ ]:
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output

import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# 01. Preprocessing

In [ ]:
train_path = '/content/MyDrive/MyDrive/Colab Notebooks/Aspect_Based_Sentiment_Analysis/Dataset/Train.csv'
test_path = '/content/MyDrive/MyDrive/Colab Notebooks/Aspect_Based_Sentiment_Analysis/Dataset/Test.csv'
dev_path = '/content/MyDrive/MyDrive/Colab Notebooks/Aspect_Based_Sentiment_Analysis/Dataset/Dev.csv'

train_data = pd.read_csv(train_path)
train_data.head()

,index,comment,n_star,date_time,label
0,0,Mới mua máy này Tại thegioididong thốt nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


In [ ]:
class_to_id = {"SCREEN": 0, "CAMERA": 1, "FEATURES": 2, "BATTERY": 3, "PERFORMANCE": 4, "STORAGE": 5, "DESIGN": 6, "PRICE": 7, "GENERAL": 8, "SER&ACC": 9}
label_to_id = {"Positive": 0, "Negative": 1, "Neutral": 2, "None": 3}

In [ ]:
def get_label_dict(sentences_label):
    label_dict = {}
    labels = sentences_label.split(';')[:-1]
    if len(labels) == 1 and labels[-1][1:-1] == "OTHERS":
        return None
    else:
        if labels[-1][1:-1] == "OTHERS":
            for l in labels[:-1]:
                class_name, sentiment = l[1:-1].split('#')
                label_dict[class_name] = sentiment
            return label_dict
        else:
            for l in labels:
                class_name, sentiment = l[1:-1].split('#')
                label_dict[class_name] = sentiment
            return label_dict

In [ ]:
sentences = train_data['comment'].tolist()
label_comment = train_data['label'].tolist()
others_label = [[0, 0, 0, 1]] * 10

train_set = []

for sentence, sentence_label in zip(sentences, label_comment):
    label_dict = get_label_dict(sentence_label)
    labels_ = []
    if label_dict:
        for aspect in list(class_to_id.keys()):
            train_label = [0] * 4
            if aspect in label_dict.keys():
                label_id = label_to_id[label_dict[aspect]]
                train_label[label_id] =  1
                labels_.append(train_label)
            else:
                labels_.append([0, 0, 0, 1])
        train_set.append(
            {'sentence': sentence, 'labels': labels_}
        )
    else:
        train_set.append(
            {'sentence': sentence, 'labels': others_label}
        )
print(train_set[0]['sentence'])
print(train_set[0]['labels'])

Mới mua máy này Tại thegioididong thốt nốt cảm thấy ok bin trâu chụp ảnh đẹp loa nghe to bắt wf khỏe sóng ổn định, giá thành vừa với túi tiền, nhân viên tư vấn nhiệt tình
[[0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]]


In [ ]:
for i, data in enumerate(train_set):
    sentence, label = data.items()
    input = np.array(label[-1])
    clear_output()
    if input.shape == (10, 4):
        print("pass")
    else:
        print(i)

pass


# 02. Build data loader

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer


class AspectSentimentDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sentence = self.data[index]['sentence']
        labels = self.data[index]['labels']

        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'sentence_text': sentence,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.float)
        }

In [ ]:
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-multilingual-cased')
dataset = AspectSentimentDataset(train_set, tokenizer, max_len=128)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
dataset.__len__()

7786

# 03. Build RNN model

In [ ]:
import torch.nn as nn
from transformers import BertModel


class AspectSentimentModel(nn.Module):
    def __init__(self, n_classes, n_labels_per_class):
        super(AspectSentimentModel, self).__init__()
        self.bert = BertModel.from_pretrained('google-bert/bert-base-multilingual-cased')
        self.rnn = nn.RNN(768, 256, batch_first=True, bidirectional=True)
        self.classifier = nn.Linear(256*2, n_classes * n_labels_per_class)

    def forward(self, input_ids, attention_mask):
        bert_outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        rnn_outputs, _ = self.rnn(bert_outputs.last_hidden_state)
        rnn_outputs = rnn_outputs[:, -1, :]
        output = self.classifier(rnn_outputs)
        return output.view(-1, 10, 4)

model = AspectSentimentModel(n_classes=10, n_labels_per_class=4)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
import torch.optim as optim


def train_epoch(model, data_loader, loss_fn, optimizer, device):
    model = model.train()
    losses = []
    correct_predictions = 0

    for data in data_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        loss = loss_fn(outputs, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return np.mean(losses)


EPOCHS = 40
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
training = True


optimizer = optim.Adam(model.parameters(), lr=2e-5)
loss_fn = nn.BCEWithLogitsLoss()

if training == True:
    for epoch in range(EPOCHS):
        train_loss = train_epoch(model, dataloader, loss_fn, optimizer, device)
        print(f'Epoch {epoch + 1}/{EPOCHS}, Train loss: {train_loss}')
        if epoch % 5 == 0:
            torch.save(model.state_dict(), f'/content/MyDrive/MyDrive/Colab Notebooks/Aspect_Based_Sentiment_Analysis/models/ABSA_using_RNN_model{epoch}.pth')

Epoch 1/40, Train loss: 0.2240360308232004
Epoch 2/40, Train loss: 0.18419618258975615
Epoch 3/40, Train loss: 0.1570158595299574
Epoch 4/40, Train loss: 0.13546707343455458
Epoch 5/40, Train loss: 0.1184748739493701
Epoch 6/40, Train loss: 0.10322257653031751
Epoch 7/40, Train loss: 0.089470876369327
Epoch 8/40, Train loss: 0.07842894155509174
Epoch 9/40, Train loss: 0.06938255990585752
Epoch 10/40, Train loss: 0.061233339991871825
Epoch 11/40, Train loss: 0.05425387879477879
Epoch 12/40, Train loss: 0.04787318307133793
Epoch 13/40, Train loss: 0.04214495317761842
Epoch 14/40, Train loss: 0.03725947599735654
Epoch 15/40, Train loss: 0.03333241639188367
Epoch 16/40, Train loss: 0.03058727961836777
Epoch 17/40, Train loss: 0.026574214520716937
Epoch 18/40, Train loss: 0.024818539384224698
Epoch 19/40, Train loss: 0.022305012746566804
Epoch 20/40, Train loss: 0.021231128175377662
Epoch 21/40, Train loss: 0.017935589674929083
Epoch 22/40, Train loss: 0.017240576302628863
Epoch 23/40, Trai

In [50]:
# path_save_model = '/content/MyDrive/MyDrive/Colab Notebooks/Aspect_Based_Sentiment_Analysis/models/ABSA_using_RNN_model.pth'
# torch.save(model.state_dict(), path_save_model+'/ABSA_using_RNN_model.pth')
torch.save(model.state_dict(), f'/content/MyDrive/MyDrive/Colab Notebooks/Aspect_Based_Sentiment_Analysis/models/ABSA_using_RNN_model{40}.pth')

In [ ]:
# model = AspectSentimentModel(n_classes=10, n_labels_per_class=4)
# model.load_state_dict(torch.load(path_save_model+'/ABSA_using_RNN_model.pth'))
# model.eval()

# 04. Predict

In [ ]:
def predict_sentiment(model, sentence, tokenizer, max_len, device):
    model = model.eval()

    encoding = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    return outputs


def interpret_predictions(predictions):
    aspect_classes = ['SCREEN', 'CAMERA', 'FEATURES', 'BATTERY', 'PERFORMANCE', 'STORAGE', 'DESIGN', 'PRICE', 'GENERAL', 'SER&ACC']
    sentiment_labels = ['Positive', 'Negative', 'Neutral', 'None']

    results = {}

    for i, aspect in enumerate(aspect_classes):
        aspect_sentiments = predictions[0, i].cpu().numpy()
        sentiment_index = aspect_sentiments.argmax()
        results[aspect] = sentiment_labels[sentiment_index]

    return results

# 05. Thực hiện predict trên toàn bộ dữ liệu kiểm thử (testset)

In [ ]:
test_data = pd.read_csv(test_path)
test_data.head()

,index,comment,n_star,date_time,label
0,0,"Điện thoải ổn. Facelock cực nhanh, vân tay ôk ...",5,5/2/2020,{SCREEN#Positive};{FEATURES#Positive};{PERFORM...
1,1,"Mình mới mua vivo91c. Tải ứng dụng ,games nh...",5,14/05/2019,{FEATURES#Negative};{PERFORMANCE#Positive};{SE...
2,2,Xấu đẹp gì ko biết nhưng rất ưng TGdđ phục vụ ...,5,26/03/2020,{DESIGN#Neutral};{SER&ACC#Positive};
3,3,Màn hình hơi lác khi chơi game. Game nặng thì ...,4,4/6/2019,{PERFORMANCE#Negative};{DESIGN#Negative};{OTHE...
4,4,"Nói chung máy đẹp với màn amoled, ổn trong tầm...",4,12/5/2020,{SCREEN#Positive};{BATTERY#Negative};{DESIGN#P...


In [ ]:
max_len = 256
idx = 20
sentence_test = test_data.loc[idx, 'comment']
true_label = test_data.loc[idx, 'label']
print(sentence_test)
print(true_label)

predictions = predict_sentiment(model, sentence_test, tokenizer, max_len, device)
results = interpret_predictions(predictions)

print('Aspect Sentiment Prediction:')
for aspect, sentiment in results.items():
    print(f'{aspect}: {sentiment}')

Mọi thứ ok.mình thấy pin trâu chứ có sao đâu.mua đc 2 tuần rồi chỉ có cảm biến vân tay ko nhạy như mong đợi thôi.
{FEATURES#Negative};{BATTERY#Positive};{GENERAL#Positive};
Aspect Sentiment Prediction:
SCREEN: None
CAMERA: None
FEATURES: Negative
BATTERY: Positive
PERFORMANCE: None
STORAGE: None
DESIGN: None
PRICE: None
GENERAL: Positive
SER&ACC: None


In [ ]:
predictions_on_test = []

for i in range(len(test_data)):
    print(i)
    sentence_on_test = test_data['comment'][i]
    prediction_on_test = predict_sentiment(model, sentence_on_test, tokenizer, max_len, device)
    results = interpret_predictions(prediction_on_test)
    predictions_on_test.append(results)
    clear_output()

print('All done!')


All done!


In [ ]:
true_labels = []

for i in range(len(test_data)):
    outcome = {}
    label = test_data['label'][i]
    label = label.replace('{', '')
    label = label.replace('}', '')
    labels = label.split(';')
    for l in labels:
        try:
            text = l.split('#')
            outcome[text[0]] = text[1]
        except:
            pass
    true_labels.append(outcome)

# 06. Chuẩn hóa nhãn đầu ra để thực hiện đánh giá hiệu suất mô hình

In [ ]:
def full_label(true_label, prediction):
    '''true_label: dict
     prediction: dict '''
    for key, value in prediction.items():
        if key not in true_label:
            true_label[key] = 'None'
    return true_label

In [ ]:
# mục tiêu là đưa danh sách các nhãn thực tế về dạng đánh giá 10 class như trong dự đoán
true_full_labels = []
for idx in range(len(predictions_on_test)):
    true_label = true_labels[idx]
    prediction = predictions_on_test[idx]
    true_full_labels.append(full_label(true_label, prediction))

In [ ]:
# data thỉnh thoảng bị sai -..-
# bằng chứng là SER&ACC trong câu này phải là Positive nhưng nhãn là None
# hoặc GENERAL nên là Positive thì nhãn lại là None

print(predictions_on_test[6])
print(true_labels[6])
print(test_data.loc[6, 'comment'])
for key, value in true_labels[6].items():
    print('true: ',key, value)
    print('predicted: ', key, predictions_on_test[6][key])

# 07. Đánh giá hiệu suất tổng thể của mô hình

In [ ]:
true_ = []
predicted_ = []

for idx in range(len(predictions_on_test)):
    for key, value in predictions_on_test[idx].items():
        if true_full_labels[idx][key] == 'None' and value == 'None':
            true_.append(0)
            predicted_.append(0)
        elif true_full_labels[idx][key] != 'None' and true_full_labels[idx][key] == value:
            true_.append(1)
            predicted_.append(1)
        elif true_full_labels[idx][key] == 'None' and value != 'None':
            true_.append(0)
            predicted_.append(1)
        elif true_full_labels[idx][key] != 'None' and value == 'None':
            true_.append(1)
            predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

# 08. Đánh giá hiệu suất nhận diện khía cạnh

### Accuracy, Precision, Recall, F1-score trên toàn bộ khía cạnh

In [ ]:
true_ = []
predicted_ = []

for idx in range(len(predictions_on_test)):
    for key, value in predictions_on_test[idx].items():
        if true_full_labels[idx] == 'None' and value == 'None':
            true_.append(0)
            predicted_.append(0)
        elif true_full_labels[idx][key] != 'None' and value != 'None':
            true_.append(1)
            predicted_.append(1)
        elif true_full_labels[idx][key] == 'None' and value != 'None':
            true_.append(0)
            predicted_.append(1)
        elif true_full_labels[idx][key] != 'None' and value == 'None':
            true_.append(1)
            predicted_.append(0)

print(f'Accuracy = {accuracy_score(true_, predicted_)}')
print(f'Precision = {precision_score(true_, predicted_)}')
print(f'Recall = {recall_score(true_, predicted_)}')
print(f'F1-score = {f1_score(true_, predicted_)}')

### đánh giá hiệu suất khía cạnh trên class SCREEN

In [27]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'SCREEN'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9518884892086331
Precision = 0.762987012987013
Recall = 0.8736059479553904
f1 = 0.8145580589254766


### đánh giá hiệu suất khía cạnh trên class FEATURES

In [28]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'FEATURES'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9132194244604317
Precision = 0.8390052356020943
Recall = 0.9015471167369902
f1 = 0.8691525423728813


### đánh giá hiệu suất khía cạnh trên class PERFORMANCE

In [29]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'PERFORMANCE'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9024280575539568
Precision = 0.9050042408821035
Recall = 0.9104095563139932
f1 = 0.9076988515525308


### đánh giá hiệu suất khía cạnh trên class GENERAL

In [30]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'GENERAL'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.8538669064748201
Precision = 0.8626373626373627
Recall = 0.9094858797972484
f1 = 0.8854423686993302


### đánh giá hiệu suất khía cạnh trên class BATTERY

In [31]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'BATTERY'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9662769784172662
Precision = 0.9458689458689459
Recall = 0.9822485207100592
f1 = 0.9637155297532656


### đánh giá hiệu suất khía cạnh trên class STORAGE

In [32]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'STORAGE'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9928057553956835
Precision = 0.6666666666666666
Recall = 0.8148148148148148
f1 = 0.7333333333333333


### đánh giá hiệu suất khía cạnh trên class DESIGN

In [33]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'DESIGN'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9514388489208633
Precision = 0.864321608040201
Recall = 0.864321608040201
f1 = 0.864321608040201


### đánh giá hiệu suất khía cạnh trên class PRICE

In [34]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'PRICE'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9482913669064749
Precision = 0.8733552631578947
Recall = 0.9332161687170475
f1 = 0.9022939677145284


### đánh giá hiệu suất khía cạnh trên class SER&ACC

In [35]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'SER&ACC'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.89568345323741
Precision = 0.8464491362763915
Recall = 0.7436762225969646
f1 = 0.7917414721723519


### đánh giá hiệu suất khía cạnh trên class CAMERA

In [36]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'CAMERA'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9734712230215827
Precision = 0.9300813008130081
Recall = 0.9727891156462585
f1 = 0.9509559434746467


# 09. Đánh giá hiệu suất nhận diện tình cảm (đa nhãn)

### Accuracy trên toàn bộ data kiểm thử
acc = corrects/(all predictions)

In [37]:
true_ = []
predicted_ = []
accuracy = []

for idx in range(len(true_full_labels)):
    for key, value in true_full_labels[idx].items():
        if value == 'None':
            continue
        elif predictions_on_test[idx][key] != value:
            accuracy.append(0)
        elif predictions_on_test[idx][key] == value:
            accuracy.append(1)

print(f'Accuracy = {sum(accuracy)/len(accuracy)}')

Accuracy = 0.8103243082415947


### Precisio, Recall, F1-score tình cảm trên toàn bộ data kiểm thử

In [38]:
from sklearn.metrics import classification_report


y_true = []
y_pred = []

for idx in range(len(true_labels)):
    for key, value in true_labels[idx].items():
        y_true.append(key + "_" + value)
        y_pred.append(key + "_" + predictions_on_test[idx][key])


print(f'Classification report:')
print(classification_report(y_true, y_pred, digits=4))

Classification report:
                      precision    recall  f1-score   support

    BATTERY_Negative     0.8293    0.9239    0.8740       368
     BATTERY_Neutral     0.4595    0.5543    0.5025        92
        BATTERY_None     0.9846    0.9529    0.9685      1210
    BATTERY_Positive     0.9154    0.8791    0.8969       554
     CAMERA_Negative     0.7656    0.8596    0.8099       171
      CAMERA_Neutral     0.5897    0.6479    0.6174        71
         CAMERA_None     0.9901    0.9737    0.9818      1636
     CAMERA_Positive     0.9217    0.9191    0.9204       346
     DESIGN_Negative     0.6250    0.5729    0.5978        96
      DESIGN_Neutral     0.4000    0.2143    0.2791        28
         DESIGN_None     0.9704    0.9704    0.9704      1826
     DESIGN_Positive     0.8373    0.9015    0.8682       274
   FEATURES_Negative     0.8033    0.8453    0.8238       459
    FEATURES_Neutral     0.3676    0.4808    0.4167        52
       FEATURES_None     0.9521    0.9187    0

In [39]:
precision_micro = precision_score(y_true, y_pred, average='micro')
recall_micro = recall_score(y_true, y_pred, average='micro')
f1_micro = f1_score(y_true, y_pred, average='micro')

precision_macro = precision_score(y_true, y_pred, average='macro')
recall_macro = recall_score(y_true, y_pred, average='macro')
f1_macro = f1_score(y_true, y_pred, average='macro')

precision_weighted = precision_score(y_true, y_pred, average='weighted')
recall_weighted = recall_score(y_true, y_pred, average='weighted')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

print("\nOverall Precision, Recall, F1-Score:")
print(f"Micro Precision: {precision_micro:.4f}")
print(f"Micro Recall: {recall_micro:.4f}")
print(f"Micro F1-Score: {f1_micro:.4f}")
print(f"Macro Precision: {precision_macro:.4f}")
print(f"Macro Recall: {recall_macro:.4f}")
print(f"Macro F1-Score: {f1_macro:.4f}")
print(f"Weighted Precision: {precision_weighted:.4f}")
print(f"Weighted Recall: {recall_weighted:.4f}")
print(f"Weighted F1-Score: {f1_weighted:.4f}")


Overall Precision, Recall, F1-Score:
Micro Precision: 0.9054
Micro Recall: 0.9054
Micro F1-Score: 0.9054
Macro Precision: 0.6952
Macro Recall: 0.7147
Macro F1-Score: 0.7017
Weighted Precision: 0.9074
Weighted Recall: 0.9054
Weighted F1-Score: 0.9057


### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh SCREEN

In [40]:
y_true = []
y_pred = []

key = 'SCREEN'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.60      0.73      0.66       116
     Neutral       0.08      0.06      0.07        17
        None       0.98      0.96      0.97      1955
    Positive       0.76      0.87      0.81       136

    accuracy                           0.94      2224
   macro avg       0.61      0.66      0.63      2224
weighted avg       0.94      0.94      0.94      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh FEATURES

In [41]:
y_true = []
y_pred = []

key = 'FEATURES'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.80      0.85      0.82       459
     Neutral       0.37      0.48      0.42        52
        None       0.95      0.92      0.94      1513
    Positive       0.70      0.74      0.72       200

    accuracy                           0.88      2224
   macro avg       0.71      0.75      0.72      2224
weighted avg       0.88      0.88      0.88      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh PERFORMANCE

In [42]:
y_true = []
y_pred = []

key = 'PERFORMANCE'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.80      0.80      0.80       454
     Neutral       0.41      0.37      0.39       116
        None       0.90      0.89      0.90      1052
    Positive       0.86      0.89      0.87       602

    accuracy                           0.85      2224
   macro avg       0.74      0.74      0.74      2224
weighted avg       0.84      0.85      0.84      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh GENERAL

In [43]:
y_true = []
y_pred = []

key = 'GENERAL'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.68      0.81      0.74       294
     Neutral       0.58      0.54      0.56        83
        None       0.84      0.76      0.80       843
    Positive       0.88      0.91      0.90      1004

    accuracy                           0.83      2224
   macro avg       0.75      0.76      0.75      2224
weighted avg       0.83      0.83      0.83      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh CAMERA

In [44]:
y_true = []
y_pred = []

key = 'CAMERA'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.77      0.86      0.81       171
     Neutral       0.59      0.65      0.62        71
        None       0.99      0.97      0.98      1636
    Positive       0.92      0.92      0.92       346

    accuracy                           0.95      2224
   macro avg       0.82      0.85      0.83      2224
weighted avg       0.95      0.95      0.95      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh BATTERY

In [45]:
y_true = []
y_pred = []

key = 'BATTERY'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.83      0.92      0.87       368
     Neutral       0.46      0.55      0.50        92
        None       0.98      0.95      0.97      1210
    Positive       0.92      0.88      0.90       554

    accuracy                           0.91      2224
   macro avg       0.80      0.83      0.81      2224
weighted avg       0.92      0.91      0.92      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh STORAGE

In [46]:
y_true = []
y_pred = []

key = 'STORAGE'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.25      0.33      0.29         6
     Neutral       0.00      0.00      0.00         3
        None       1.00      0.99      1.00      2197
    Positive       0.56      0.78      0.65        18

    accuracy                           0.99      2224
   macro avg       0.45      0.53      0.48      2224
weighted avg       0.99      0.99      0.99      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh DESIGN

In [47]:
y_true = []
y_pred = []

key = 'DESIGN'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.62      0.57      0.60        96
     Neutral       0.40      0.21      0.28        28
        None       0.97      0.97      0.97      1826
    Positive       0.84      0.90      0.87       274

    accuracy                           0.94      2224
   macro avg       0.71      0.66      0.68      2224
weighted avg       0.93      0.94      0.93      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh PRICE

In [48]:
y_true = []
y_pred = []

key = 'PRICE'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.47      0.66      0.55        79
     Neutral       0.78      0.76      0.77       328
        None       0.98      0.95      0.96      1655
    Positive       0.73      0.80      0.76       162

    accuracy                           0.90      2224
   macro avg       0.74      0.79      0.76      2224
weighted avg       0.91      0.90      0.91      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh SER&ACC

In [49]:
y_true = []
y_pred = []

key = 'SER&ACC'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.61      0.44      0.51       167
     Neutral       0.15      0.15      0.15        27
        None       0.91      0.95      0.93      1631
    Positive       0.87      0.82      0.84       399

    accuracy                           0.88      2224
   macro avg       0.64      0.59      0.61      2224
weighted avg       0.87      0.88      0.87      2224

